In [ ]:
# most code sourced from https://www.projectpro.io/article/python-chatbot-project-learn-to-build-a-chatbot-from-scratch/429
# dataset created from scratch

from google.colab import drive
drive.mount('/content/drive')
data_root = '/content/drive/My Drive/emotionschatbot'

import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras.models import Sequential
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense, Dropout
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

data_file = open(data_root + '/intents.json').read()
data = json.loads(data_file)

words = []
classes = []
data_x = []
data_y = []

for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern)
    words.extend(tokens)
    data_x.append(pattern)
    data_y.append(intent["tag"]),
  if intent["tag"] not in classes:
    classes.append(intent["tag"])

lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation]
words = sorted(set(words))
classes = sorted(set(classes))

training = []
out_empty = [0] * len(classes)
for idx, doc in enumerate(data_x):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)
  output_row = list(out_empty)
  output_row[classes.index(data_y[idx])] = 1
  training.append([bow, output_row])
random.shuffle(training)
train_x = np.array([np.array(x).flatten() for x in np.array(training)[:,0]])
train_y = np.array([np.array(y).flatten() for y in np.array(training)[:,1]])
x = tf.convert_to_tensor(train_x, dtype=tf.float32)
y = tf.convert_to_tensor(train_y, dtype=tf.float32)

from tensorflow.python.types.core import Tensor
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
adam = tf.keras.optimizers.Adam(learning_rate=0.01, epsilon=1e-6)
model.compile(loss='categorical_crossentropy',
              optimizer=adam,
              metrics=["accuracy"])
print(model.summary())
model.fit(x, y, epochs=150, verbose=1)

def clean_text(text):
  tokens = nltk.word_tokenize(text)
  tokens = [lemmatizer.lemmatize(word) for word in tokens]
  return tokens

def bag_of_words(text, vocab):
  tokens = clean_text(text)
  bow = [0] * len(vocab)
  for w in tokens:
    for idx, word in enumerate(vocab):
      if word == w:
        bow[idx] = 1
  return np.array(bow)

def pred_class(text, vocab, labels):
  bow = bag_of_words(text, vocab)
  result = model.predict(np.array([bow]))[0]
  thresh = 0.5
  y_pred = [[indx, res] for indx , res in enumerate(result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True)
  return_list = []
  for r in y_pred:
    return_list.append(labels[r[0]])
  return return_list

def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Sorry! I do not understand."
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
      if i["tag"] == tag:
        result = random.choice(i["responses"])
        break
  return result

print('Hi! My name is CareBot, and I would love to hear about your day! If you have urgent needs, please respond with "urgent" and you will be redirected. To exit, please respond with"0".')
while True:
  message = input("")
  if message == "0":
    break
  if message is not None:
    intents = pred_class(message, words, classes)
    result = get_response(intents, data)
    print(result)